In [21]:
!pip install psycopg2
import psycopg2 as ps
import pandas as pd

Importing Dataframe to Database
1. Connect to the database
2. Create table 
3. Import dataframe
    

1. Connecting to database

In [32]:
def connect_2_database(host_name, dbname, username, password, port):
    try:
        conn = ps.connect(host=host_name, database=dbname, user=username, password=password, port=port)
    except ps.OperationalError as e:
        raise e
    else:
        print("Connected")
        return conn

2. Creating Table

In [24]:
#write out the SQL command line to create a table
def create_table(curr):
    create_table_command = ("""CREATE TABLE IF NOT EXISTS VIDEOS (
        VIDEO_ID VARCHAR(255) PRIMARY KEY,
        VIDEO_TITLE TEXT NOT NULL,
        UPLOAD_DATE DATE NOT NULL DEFAULT CURRENT_DATE,
        VIEW_COUNT INTEGER NOT NULL,
        LIKE_COUNT INTEGER NOT NULL,
        DISLIKE_COUNT INTEGER NOT NULL,
        COMMENT_COUNT INTEGER NOT NULL
    )   """)
    curr.execute(create_table_command)

In [39]:
#check to see if video exists
def check_if_video_exists(curr,video_id):
    query = ("""SELECT video_id FROM VIDEOS WHERE video_id = %s""")
    curr.execute(query, (video_id,))

    return curr.fetchone() is not None

In [34]:
#update row function
def update_row(curr, video_title, view_count, like_count, dislike_count, comment_count, video_id):
    query = ("""UPDATE videos
                SET video_title = %s,
                view_count = %s,
                like_count = %s,
                dislike_count = %s,
                comment_count = = %s
                WHERE video_id = %s;""")

                # all the columns in the pandas dateframe               
    columns = (video_title, view_count, like_count, dislike_count, comment_count, video_id)
    curr.execute(query,columns)


In [41]:
#updating the database
#handling data for scalability
def update_db(curr, df):
    temp_df = pd.DataFrame(columns=['video_id','video_title', 'upload_date', 'view_count', 'like_count', 'dislike_count', 'comment_count'])

    #check to see if video exists
    for i, row in df.iterrows():
        #if video already exists then we will update
        if check_if_video_exists(curr,row['video_id']):
            update_row(curr,row['video_title'], row['view_count'], row['like_count'], row['dislike_count'], row['comment_count'], row['video_id']) #update SQL command
        #else the video doesnt exist so we will append to the db table    
        else:
            temp_df = temp_df.append(row) #insert command
    return temp_df        

In [45]:
#write insert command
def insert_into_table(curr,video_id,video_title,upload_date,view_count,like_count,dislike_count,comment_count):
    insert_to_db = ("""INSERT INTO videos (video_id, video_title, upload_date, view_count, like_count, dislike_count, comment_count)
                    VALUES (%s,%s, %s, %s, %s, %s, %s); """)

    rows_to_insert = (video_id,video_title,upload_date,view_count,like_count,dislike_count,comment_count)
    curr.execute(insert_to_db, rows_to_insert)

In [48]:
def append_data_to_db(curr, df):
    for i, row in df.iterrows():
        insert_into_table(curr, row['video_id'], row['video_title'],row['upload_date'],row['view_count'],row['like_count'], row['dislike_count'],row['comment_count']) 

In [2]:
df = pd.read_csv("Youtube_Data.csv", index_col=0)
df.head()

,video_id,video_title,upload_date,view_count,like_count,dislike_count,comment_count
0,rUICJO4qwms,How To Grow Your YouTube Channel - 5 Tips For ...,2021-10-26,93860,2879,34,447
1,wbBY2tTqXDA,Relations and Functions,2021-08-25,67543,1863,21,128
2,dHjWVlfNraM,Kinematics In One Dimension - Physics,2021-05-31,212740,4013,57,161
3,OqsOIp5UuME,Resonance Structures,2021-05-24,101855,1500,34,67
4,zRKZ0-kOUZM,Geometric Series and Geometric Sequences - Bas...,2021-05-16,175094,3125,77,109


In [29]:
host_name = 'yt-database.canwxyhiqzan.us-east-2.rds.amazonaws.com'
dbname = ''
port = '5432'
username = 'postgres'
password = 'Mmaxierucky100'
conn = None 

conn = connect_2_database(host_name, dbname, username, password, port)

Connected


In [30]:
# this cursor function lets us execute SQL commands in python
curr = conn.cursor()

In [26]:
create_table(curr)

In [42]:
#We save the results of update_db() to a variable beacuse it'll return a df
# with the new videos we'll have to insert into our database.  
new_videos_df = update_db(curr, df)

0
video_id                                               rUICJO4qwms
video_title      How To Grow Your YouTube Channel - 5 Tips For ...
upload_date                                             2021-10-26
view_count                                                   93860
like_count                                                    2879
dislike_count                                                   34
comment_count                                                  447
Name: 0, dtype: object
1
video_id                     wbBY2tTqXDA
video_title      Relations and Functions
upload_date                   2021-08-25
view_count                         67543
like_count                          1863
dislike_count                         21
comment_count                        128
Name: 1, dtype: object
2
video_id                                   dHjWVlfNraM
video_title      Kinematics In One Dimension - Physics
upload_date                                 2021-05-31
view_count                 

In [43]:
conn.commit()

In [44]:
new_videos_df

,video_id,video_title,upload_date,view_count,like_count,dislike_count,comment_count
0,rUICJO4qwms,How To Grow Your YouTube Channel - 5 Tips For ...,2021-10-26,93860,2879,34,447
1,wbBY2tTqXDA,Relations and Functions,2021-08-25,67543,1863,21,128
2,dHjWVlfNraM,Kinematics In One Dimension - Physics,2021-05-31,212740,4013,57,161
3,OqsOIp5UuME,Resonance Structures,2021-05-24,101855,1500,34,67
4,zRKZ0-kOUZM,Geometric Series and Geometric Sequences - Bas...,2021-05-16,175094,3125,77,109
5,XZJdyPkCxuE,Arithmetic Sequences and Arithmetic Series - B...,2021-05-13,287783,5852,87,192
6,J6TnZxUUzqU,Complex Numbers In Polar - De Moivre&#39;s The...,2021-05-10,111724,1938,38,110
7,KYgmOTLbuqE,Finding The Focus and Directrix of a Parabola ...,2021-05-02,203797,3381,67,144
8,OPSCKXXvWiM,Writing Equations of Ellipses In Standard Form...,2021-04-29,149027,2595,52,97
9,QKkdYW77xNI,Partial Fraction Decomposion,2021-04-23,272121,4359,97,133


In [49]:
append_data_to_db(curr, new_videos_df)